In [1]:
from queue import Queue
from threading import Thread
from flask import Flask, request
from telegram.ext import MessageHandler, Filters

from telegram import Bot,Update
from telegram.ext import Dispatcher
import logging
import sys
import os

file_handler = logging.FileHandler(filename='log.txt',encoding='utf-8')
stdout_handler = logging.StreamHandler(sys.stdout)
handlers = [file_handler, stdout_handler]

logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO,handlers=handlers)

logger = logging.getLogger(__name__)

TOKEN = "1862633589:AAE9TUHq4X_ifyM1GX1hXZ15bFWeIuKUOc8"
NAME = "telegramchatbott"
PORT = int(os.environ.get('PORT', '8443'))

bot = Bot(TOKEN)

def echo(update, context):
    text = update.message.text
    update.message.reply_text(text)

def setup(TOKEN):
    # update queue and dispatcher instances
    update_queue = Queue()

    dispatcher = Dispatcher(bot, update_queue, use_context=True)

    ##### Register handlers here #####
    echo_handler = MessageHandler(Filters.text, echo)
    dispatcher.add_handler(echo_handler)

    # Start the thread
    bot.setWebhook("https://{}.herokuapp.com/{}".format(NAME, TOKEN))
    thread = Thread(target=dispatcher.start, name='dispatcher')
    thread.start()

    return update_queue
    # you might want to return dispatcher as well,
    # to stop it at server shutdown, or to register more handlers:
    # return (update_queue, dispatcher)

app = Flask(__name__)
update_queue = setup(TOKEN)

@app.route('/')
def hello_world():
    return 'Hello World!'

@app.route('/' + TOKEN, methods=['GET','POST'])
def pass_update():
    new_update = Update.de_json(request.get_json(force=True),bot)
    update_queue.put(new_update)
    return "ok"

if __name__ == '__main__':
    app.run(threaded=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
2021-05-12 04:03:12,480 - werkzeug - INFO -  * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
2021-05-12 04:03:20,447 - werkzeug - INFO - 127.0.0.1 - - [12/May/2021 04:03:20] "GET / HTTP/1.1" 200 -
2021-05-12 04:03:21,290 - werkzeug - INFO - 127.0.0.1 - - [12/May/2021 04:03:21] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:
from flask import Flask
from flask import render_template, request
import logging
import telegram
import os
import requests

HOST = "https://telegramchatbott.herokuapp.com/"

app = Flask(__name__)
logging.basicConfig(level=logging.DEBUG,format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

global bot
bot = telegram.Bot(token='1862633589:AAE9TUHq4X_ifyM1GX1hXZ15bFWeIuKUOc8')
botName = "Zam" #Without @


@app.route("/", methods=["POST", "GET"])
def setWebhook():
    if request.method == "GET":
        logging.info("Hello, Telegram!")
        print ("Done")
        return "OK, Telegram Bot!"
    
    

@app.route("/verify", methods=["POST"])
def verification():
    if request.method == "POST":
        update = telegram.Update.de_json(request.get_json(force=True),bot)
        if update is None:
            return "Show me your TOKEN please!"
        logging.info("Calling {}".format(update.message))
        handle_message(update.message)
        return "ok"


def handle_message(msg):
    text = msg.text
    print (msg)
    # An echo bot
    bot.sendMessage(chat_id=msg.chat.id, text=text)


if __name__ == "__main__":
    s = bot.setWebhook("{}/verify".format(HOST))
    if s:
        logging.info("{} WebHook Setup OK!".format(botName))
    else:
        logging.info("{} WebHook Setup Failed!".format(botName))
    app.run(threaded=True)